In [50]:
#First we load the data and packages.
import pandas as pd
import numpy as np
from copy import deepcopy
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#import xgboost as xgb
#from xgboost.xgbclassifier import XGBClassifier as xgb

In [41]:

#data_path = pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv",index_col=0)'
#train_path = data_path + 'u1.base'
#test_path = data_path + 'u1.test'

# load train and test data, make dummies from genres and combine it with df.
df1 = pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv",index_col=0)
genres = df.genres.str.get_dummies(sep = "|")

df = pd.concat([df1, genres], axis=1)
df = df.drop(['genres'], axis = 1)

In [44]:
data = df.drop(['rating', 'title'], axis= 1)
target = df[['rating']]

In [45]:
#split df in training and test data
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state= 101)

In [48]:
model1 = xgb.XGBRegressor(objective='reg:squarederror')
model1.fit(x_train, y_train)

pred1 = model1.predict(X_test)
rmse = np.sqrt(np.mean((pred1 - y_test.to_numpy())**2))
print(f'content-based rmse = {rmse}')

NameError: name 'xgb' is not defined

array(['Adventure|Animation|Children|Comedy|Fantasy', 'Comedy|Romance',
       'Action|Crime|Thriller', 'Mystery|Thriller',
       'Crime|Mystery|Thriller', 'Action|Comedy|Horror|Thriller',
       'Adventure|Comedy|Crime|Romance', 'Action|Drama|War',
       'Action|Drama|Romance|War', 'Comedy|War', 'Action|Romance|Western',
       'Comedy', 'Adventure|Comedy', 'Comedy|Drama',
       'Action|Adventure|Sci-Fi', 'Comedy|Crime|Drama|Thriller',
       'Action|Crime|Drama|Thriller', 'Comedy|Drama|Romance|War',
       'Adventure|Children|Romance', 'Action|Comedy|Crime|Fantasy',
       'Action|Thriller', 'Thriller', 'Action|Adventure|Sci-Fi|Thriller',
       'Drama|War', 'Comedy|Romance|Thriller',
       'Action|Adventure|Comedy|Romance', 'Action|Drama|Western',
       'Adventure|Drama|Western', 'Crime|Horror|Thriller',
       'Animation|Children|Fantasy|Musical',
       'Action|Adventure|Mystery|Thriller',
       'Adventure|Animation|Children|Fantasy|Musical',
       'Adventure|Animation|Chil

Next, we perform the feature engineering on our user and movie dataset to fit the model, such that
-One-hot Encoding on the user’s occupation and movie’s genre
-Label Encoding on the user’s gender